In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import math
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import wandb as wb
import gc
from tqdm import tqdm
import time

In [2]:
# api = 7ef702aa33359df589bdaccee0566a96d8c08d31

In [1]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip

--2024-04-06 20:28:52--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.207, 172.253.119.207, 108.177.121.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: 'nature_12K.zip'

nature_12K.zip      100%[===================>]   3.55G   195MB/s    in 18s     

2024-04-06 20:29:10 (201 MB/s) - 'nature_12K.zip' saved [3816687935/3816687935]



In [4]:
!unzip -q nature_12K.zip

In [3]:
# !rm -r inaturalist_12K

In [7]:
class CNNArchitecture(nn.Module):
    def __init__(self,param,h,w):
        super(CNNArchitecture,self).__init__()
        
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, param.filters[0], param.filterSize)
        if param.batchNormalization==True:
            self.batchnorm1 = nn.BatchNorm2d(param.filters[0])
        self.conv2 = nn.Conv2d(param.filters[0],param.filters[1],param.filterSize)
        if param.batchNormalization==True:
            self.batchnorm2 = nn.BatchNorm2d(param.filters[1])
        self.conv3 = nn.Conv2d(param.filters[1],param.filters[2],param.filterSize)
        if param.batchNormalization==True:
            self.batchnorm3 = nn.BatchNorm2d(param.filters[2])
        self.conv4 = nn.Conv2d(param.filters[2],param.filters[3],param.filterSize)
        if param.batchNormalization==True:
            self.batchnorm4 = nn.BatchNorm2d(param.filters[3])
        self.conv5 = nn.Conv2d(param.filters[3],param.filters[4],param.filterSize)
        if param.batchNormalization==True:
            self.batchnorm5 = nn.BatchNorm2d(param.filters[4])
        
        self.flatten_features =None
        #we need flatten features as an input for first dense layers without this our model will not be compatible
        # we are sending dummy image to our cnn layers and calculating what will be the parameters of it
        self.calculateFeatures(param,torch.rand(1,3,h,w))
        self.linearLayers = nn.ModuleList()

        #TODO: I didn't added activation layers here I have to do this work in forward pass
        if param.denseLayers!=0:
            self.linearLayers.append(nn.Linear(self.flatten_features,param.denseLayersSize))
            for _ in range(param.denseLayers-1):
                if int(param.dropout)!=0:
                    self.linearLayers.append(nn.Dropout(param.dropout))
                self.linearLayers.append(nn.Linear(param.denseLayersSize,param.denseLayersSize))
                
            self.linearLayers.append(nn.Linear(param.denseLayersSize,10))
        else:
            self.linearLayers.append(nn.Linear(param.flatten_features,10))

    def calculateFeatures(self,param,x):
        z = param.poolingSize
        activation = param.activation
        print(z)
        x = F.max_pool2d(activation(self.conv1(x)), z)
        print(x.size())
        x = F.max_pool2d(activation(self.conv2(x)), z)
        print(x.size())
        x = F.max_pool2d(activation(self.conv3(x)),z)
        print(x.size())
        x = F.max_pool2d(activation(self.conv4(x)),z)
        print(x.size())
        x = F.max_pool2d(activation(self.conv5(x)),z)
        print(x.size())
        self.flatten_features = x.size(1) * x.size(2) * x.size(3)
        
        
        
    def forward(self,param, x):
        z = param.poolingSize
        activation = param.activation
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(activation(self.conv1(x)), z)
        if param.batchNormalization==True:
            x = self.batchnorm1(x)
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(activation(self.conv2(x)), z)
        if param.batchNormalization == True:
            x = self.batchnorm2(x)
        x = F.max_pool2d(activation(self.conv3(x)),z)
        if param.batchNormalization == True:
            x = self.batchnorm3(x)
        x = F.max_pool2d(activation(self.conv4(x)),z)
        if param.batchNormalization == True:
            x = self.batchnorm4(x)
        x = F.max_pool2d(activation(self.conv5(x)),z)
        if param.batchNormalization == True:
            x = self.batchnorm5(x)
        
        
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension 
        for i in range(len(self.linearLayers)-1):
            x = activation(self.linearLayers[i](x))
        x = self.linearLayers[-1](x)
        return x

In [8]:
class Parameters:
    def __init__(self,filters,filter_size,pooling_size,stride,multiplier,dense_layers,dense_layer_size,aug,normalization,dropout,activation,optimizers,lr):
        self.cnnLayers = 5
        self.filterMultiplier = multiplier# number float
        self.filters = self.settingFilters(filters,multiplier,self.cnnLayers)
        self.filterSize = filter_size
        self.poolingSize = pooling_size
        self.stride = stride
        self.denseLayers = dense_layers
        self.denseLayersSize = dense_layer_size
        self.dataAugmentation = aug
        self.batchNormalization = normalization# true or false
        self.dropout = dropout # probabilty
        self.activation_dict = { 'relu':F.relu,'selu':F.selu,'gelu':F.gelu,'mish':F.mish}
        self.optimzers_dict = {'sgd':optim.SGD,'nadam':optim.NAdam,"adam":optim.Adam,"rmsprop":optim.RMSprop}
        self.activation = self.activation_dict[activation]
        self.optimizer = self.optimzers_dict[optimizers]
        self.learning_rate = lr
        
    def settingFilters(self,filters,multiplier,layers):
        return [filters*(multiplier**i) for i in range(layers)]

In [9]:
def get_default_device():
    """ Set Device to GPU or CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    

def to_device(data, device):
    "Move data to the device"
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking = True)

class DeviceDataLoader():
    """ Wrap a dataloader to move data to a device """
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        """ Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b,self.device)
            
    def __len__(self):
        """ Number of batches """
        return len(self.dl)

In [10]:
data_dir = 'inaturalist_12K/train'
test_data_dir = 'inaturalist_12K/val'
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((512,512)),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))
val_dataset = ImageFolder(test_data_dir,transforms.Compose([
    transforms.Resize((512,512)),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))
batch_size = 64
train_dl = DataLoader(dataset, batch_size, shuffle = True,pin_memory=True, num_workers = 2)
val_dl = DataLoader(val_dataset, batch_size,shuffle = True,pin_memory = True, num_workers = 2)
device = get_default_device()
print(device)

cuda


In [8]:
train_dl_gpu = DeviceDataLoader(train_dl,device)
val_dl_gpu = DeviceDataLoader(val_dl,device)

In [9]:
img,labels = next(iter(train_dl))
height = img[0].size(1)
width = img[0].size(2)
print(height,width)

512 512


In [10]:
# filters = 48 # filters are here represent number of filters in cnn 
# filter_size = 5
# pooling_size = 2
# stride=2
# multiplier = 2
# dense_layers = 5
# dense_size = 256
# aug = False
# normalization = True
# dropout = 0.25
# activation = 'relu'
# optimizers = 'adam'
# ob = Parameters(filters,filter_size,pooling_size,stride,multiplier,dense_layers,dense_size,aug,normalization,dropout,activation,optimizers)

In [10]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [11]:
def evaluate(model, ob ,dataset_tensor, datatype ,use_cuda = True):
    model.eval()
    correct = 0
    total = 0
    total_loss  = []
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for data in dataset_tensor:
            images, labels = data
           # print(images.device)
            outputs = model.forward(ob,images)
#             loss+=F.cross_entropy(outputs,labels)
            loss= criterion(outputs,labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_loss.append(loss)
    loss = torch.stack(total_loss).mean().item()
    acc = (100*correct/total)
    print(f'{datatype}_accuracy: {acc}, {datatype}_loss: {loss}')
    wb.log({f'{datatype}_accuracy': acc})
    wb.log({f'{datatype}_loss': loss})

    
# this function will also work without gpu
def fit(ob,model,train_gpu,val_gpu,epochs):
    optimizer = ob.optimizer(model.parameters(),lr = ob.learning_rate)
    history =[]
    for i in range(epochs):
        model.train()
#         training_loss = []
        acc =[]
        for ind, (images, labels) in enumerate(tqdm(train_gpu, desc=f'Training Progress {i+1}')):
            optimizer.zero_grad()
            pred = model.forward(ob,images)
        
            loss = F.cross_entropy(pred, labels)
#             training_loss.append(loss)
            loss.backward()
            optimizer.step()
        training_acc = evaluate(model,ob,train_gpu,'training')
        validation_acc = evaluate(model,ob,val_gpu,'validation')


In [12]:
epochs =10

In [13]:
def main():
    wb.init(project="Assignment 2 cnn")
    config = wb.config
    run_name = f'{config.optimizer}_{config.activation}_{config.filters}_{config.normalization}_{config.multiplier}_{config.filter_size}'

    # Set the run name
    wb.run.name = run_name
    wb.run.save()
   # epochs = config.epochs
    aug = False
    # Define and train the model as before
    ob = Parameters(config.filters,config.filter_size,config.pooling_size,config.stride,config.multiplier,config.dense_layer,config.dense_size,aug,config.normalization,config.dropout,config.activation,config.optimizer,config.learning_rate)
    model = CNNArchitecture(ob,height,width)
#     if torch.cuda.is_available():
#     gpu_model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)
    if torch.cuda.is_available():
        model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)
        optimizer = ob.optimizer(model.parameters(),lr = ob.learning_rate)
        history =[]
        for i in range(epochs):
            model.train()
#         training_loss = []

            for ind, (images, labels) in enumerate(tqdm(train_dl_gpu, desc=f'Training Progress {i+1}')):
                optimizer.zero_grad()
                pred = model.forward(ob,images)
        
                loss = F.cross_entropy(pred, labels)
#             training_loss.append(loss)
                loss.backward()
                optimizer.step()
            training_acc = evaluate(model,ob,train_dl_gpu,'training')
            validation_acc = evaluate(model,ob,val_dl_gpu,'validation')
        #fit(ob,model,train_dl_gpu,val_dl_gpu,10)
        model.cpu()
        del model
        gc.collect()
        torch.cuda.empty_cache()

In [14]:
#filters,filter_size,pooling_size,stride,multiplier,learning_rate,dense_layers,dense_size,aug,normalization,dropout,activation,optimizers
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep cross entropy',
    'metric': {
      'name': 'validation_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'filters': {
          'values': [32,128,64,16]
        },
        'filter_size': {
          'values': [3,5]
        },
        'multiplier':{
            'values':[1,2]
        },
        'pooling_size':{
            'values':[2]
        },
        'stride':{
            'values':[1,2]
        },
        'learning_rate': {
            'values':[1e-3,1e-4]
        },
        'dense_layer':{
            'values': [1,2,3]
        },
        'dense_size':{
            'values':[128,256,512]
        },
        
        'normalization':{
            'values': [True,False]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'activation': {
            'values': ['relu', 'selu','mish','gelu']
        },
        'optimizer': {
            'values': ['nadam', 'adam','sgd','rmsprop']
        }
    }
}


sweep_id = wb.sweep(sweep=sweep_config,project='Assignment 2 cnn')
wb.agent("3q087yux"  , function = main , count = 1)
wb.finish()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 04ajajzq
Sweep URL: https://wandb.ai/deeplearning-assignment/Assignment%202%20cnn/sweeps/04ajajzq


wandb: Agent Starting Run: okr276qn with config:
wandb: 	activation: relu
wandb: 	dense_layer: 1
wandb: 	dense_size: 128
wandb: 	dropout: 0
wandb: 	filter_size: 3
wandb: 	filters: 128
wandb: 	learning_rate: 0.0001
wandb: 	multiplier: 2
wandb: 	normalization: False
wandb: 	optimizer: adam
wandb: 	pooling_size: 2
wandb: 	stride: 1
wandb: Currently logged in as: cs23m023 (deeplearning-assignment). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


2
torch.Size([1, 128, 255, 255])
torch.Size([1, 256, 126, 126])
torch.Size([1, 512, 62, 62])
torch.Size([1, 1024, 30, 30])
torch.Size([1, 2048, 14, 14])


Training Progress 1:   0%|          | 0/157 [00:06<?, ?it/s]


Run okr276qn errored:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_34/181686893.py", line 26, in main
    pred = model.forward(ob,images)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py", line 185, in forward
    outputs = self.parallel_apply(replicas, inputs, module_kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py", line 200, in parallel_apply
    return parallel_apply(replicas, inputs, kwargs, self.device_ids[:len(replicas)])
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 110, in parallel_apply
    output.reraise()
  File "/opt/conda/lib/python3.10/site-packages/torch/_utils.py", line 694, in reraise
    raise exception
RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/opt/co

In [ ]:
fit(ob,gpu_model,train_dl,val_dl,50)

doing garbage collection 

In [37]:


from numba import cuda
device = cuda.get_current_device()
device.reset()